In [1]:
from anytree import Node, RenderTree, PreOrderIter
import anytree
import random
import pandas as pd
import numpy as np
import copy



import seaborn as sns
import matplotlib.pyplot as plt
import sys
print(sys.getrecursionlimit())
%matplotlib inline



'''
Test data 2:

classes = [0, 1]
feature_names = ["x","y","z","class"]
class_attributes = [[10,12,8],[12,8,10],[8,10,12]]

# Creates random data with class, class atrribute 2d array and number of datapoints per class
def create_random_data(classes, class_attributes,n):
    data_points = []
    for i in range(0,n):
        for class_type in classes:
            new_data_point = []
            for attribute in class_attributes[class_type]:
                new_data_point.append(random.gauss(attribute,1.5))
            new_data_point.append(class_type)
            data_points.append(new_data_point)
    return data_points
    
# Create test data and transform it into a data frame
data = create_random_data(classes, class_attributes,100)
X = to_data_frame(data, feature_names)



'''

# creates a data frame from a list of lists
def to_data_frame(data, feature_names):
    return pd.DataFrame(data, columns = feature_names)

#split data into decision sets
def split_data(feature, operator, value, dataFrame):
    try:
        leftDataFrame = dataFrame.query('{feature} {operator} {value}'.format(feature = feature, operator = operator, value = str(value)))
        rightDataFrame = dataFrame[~dataFrame.isin(leftDataFrame)].dropna().reset_index(drop=True)
        leftDataFrame.reset_index(drop=True,inplace=True)
        return leftDataFrame, rightDataFrame
    except Exception as e:
        print(e, feature, operator, value)
        return None

# gets all features of the dataset, removing class
def get_features(dataFrame):
    features = []
    for column in dataFrame.columns:
        if column != "Class":
            features.append(column)
    return features

# Gets classes that are available in a data frame
def get_classes(dataFrame):
    return dataFrame["Class"].unique()

# untested
def get_classes_multi(dataFrame, *dataFrames):
    combinedDataFrame = reduce(lambda left,right: pd.merge(left,right,on='name'), [dataFrame] + [dataFrames])
    return get_classes(combinedDataFrame)
    


def create_simulated_data(classes, n, d):
    data_points = []
    for i in range(0,n):
        newClass = np.random.binomial(1,0.5)
        new_data_point = []
        if(newClass == 0):
            # x axis
            new_data_point.append(np.random.uniform(0,0.5,1)[0].round(3))
            # y axis
            new_data_point.append(np.random.uniform(0,0.5,1)[0].round(3))
                
        else:
            new_data_point.append(np.random.uniform(0,1,1)[0].round(3))
            if(new_data_point[0] <= 0.5):
                new_data_point.append(np.random.uniform(.51,1,1)[0].round(3))
            else:
                new_data_point.append(np.random.uniform(0,1,1)[0].round(3))
        new_data_point += [1 for i in range(0,d-2)]
        new_data_point.append(newClass)
        data_points.append(new_data_point)
    return data_points


n = 100
d = 3
simulated = create_simulated_data([0,1],n,d)
simulatedData = to_data_frame(simulated, ["x_{n}".format(n = n) for n in range(0,d)]+["Class"])
test_data = pd.read_csv('Iris.csv')
del test_data["Id"]
#test_data.plot(x="SepalLengthCm",y="SepalWidthCm", hue="Species",kind="scatter")
#plt.show()

test_data.rename(columns={'Species': 'Class'},inplace="True")


3000


In [2]:
# Calculates the error of a dataframe given a target class
def calculate_combined_error(dataFrame, targetClass):
    return len(dataFrame.query("Class != @targetClass").index)

# Calculates the error of a dataframe using the most popular class
def popular_error(dataFrame):
    return calculate_combined_error(dataFrame, dataFrame["Class"].value_counts().first_valid_index())

# Calculates the error value of how well the tree fits the data
# Input: Tree, Dataframe
# Recursive function
def error_value(tree, dataFrame):
    # misclassification error
    error = 0
    if not tree.is_leaf:
        
        # split data by decision
        feature, operator, value = tree.name.split()
        leftData,rightData  = split_data(feature, operator, value, dataFrame)
        # recure through left and right nodes
        if((not leftData.empty) and not tree.children[0].right):
            error += error_value(tree.children[0],leftData)
        if((not rightData.empty) and len(tree.children)>1):
            error += error_value(tree.children[1],rightData)
        elif((not rightData.empty) and tree.children[0].right):
            error += error_value(tree.children[0],rightData)
    else:
        '''# Not actually passing the right values through
        #print(tree.Class,tree.error,len(dataFrame.query("Class != '{treeClass}'".format(treeClass = tree.Class)).index),len(data_values(tree,test_data).index))
        if(tree.error != len(dataFrame.query("Class != '{treeClass}'".format(treeClass = tree.Class)).index)):
            print(tree, dataFrame.query("Class != '{treeClass}'".format(treeClass = tree.Class)).index)
        error = tree.error'''
        error = len(dataFrame.query("Class != '{treeClass}'".format(treeClass = tree.Class)).index)
        
    return error



# Calculates the penalizing value for the complexity of the tree
# Input: Tree
# Recursive function
def complexity(tree):
    length = 0
    if not tree.is_leaf:
        # increase complexity by number of splits in the tree
        length += 1
        for child in tree.children:
            length += complexity(child)
    return length


# calculates the loss function for the tree and the respective data
def loss_function(tree, dataFrame, alpha = 0.5,baseLineError = 1):
    if baseLineError <= 0:
        baseLineError = 1
    return (1/baseLineError) * error_value(tree, dataFrame) + alpha * complexity(tree)

In [47]:
# Create a leaf
# Uses most populous class in available data frame
def create_leaf(dataFrame,parent=None,right=False):
    
    if not dataFrame.empty:
        
        # Get the most populous class
        Class = dataFrame['Class'].value_counts().first_valid_index()

        # Get accuracy of class
        accuracy = (dataFrame['Class'].value_counts().iloc[0]/len(dataFrame.index)).round(3)
        error = calculate_combined_error(dataFrame, Class)
        size = len(dataFrame.index)
        
        # Create Node using parent node
        leaf = Node("class = {Class}, accuracy = {accuracy}, size = {size} error = {error}".format(Class=Class,accuracy=accuracy,size = size, error = error),parent=parent, Class = Class, accuracy = accuracy, error = error,size = size, right = right, checked = False)

        return leaf
    else:
        return None
    
# recursive function for creating trees
def create_tree(dataFrame, features, parent = None, remainingLength = 0, right = False, operators = ["<"], n_min = 1):
    
    # create node with splits
    if(remainingLength > 0 and len(dataFrame.index) >= n_min * 2):
        
        
        leftDataFrame, rightDataFrame = pd.DataFrame(), pd.DataFrame()
        # Restrict possible dataframes by minimum data points
        while len(leftDataFrame.index) < n_min or len(rightDataFrame.index) < n_min:
            # split decision
            ############################################
            # Pick random feature and operator
            feature, operator = random.choice(features), random.choice(operators)
            # Choose random value in available feature
            value = random.choice(dataFrame[feature])
            ############################################
            # Create decision node

            

            # Split dataset on decision
            leftDataFrame, rightDataFrame = split_data(feature, operator, value, dataFrame)
            
        # Create decision node
        tree = Node('{feature} {operator} {value}'.format(feature = feature,operator = operator,value = value),parent = parent, right = right, checked = False)
        
        # create child nodes of decision node
        if(not leftDataFrame.empty):
            create_tree(leftDataFrame,features,tree,remainingLength=remainingLength-1,right=False, n_min = n_min)
        if(not rightDataFrame.empty):
            create_tree(rightDataFrame,features,tree,remainingLength=remainingLength-1,right=True, n_min = n_min)
    # Create a class node when at max length or no possible splits remaining
    else:
        # create leaf
        tree = create_leaf(dataFrame, parent, right)
    return tree

# prints the tree in ascii
def print_tree(tree):
    for pre, fill, node in RenderTree(tree):
        print("%s%s, %s, %s" % (pre, node.name,"Left" if not node.right else "Right", node.checked))
        


In [50]:
# Trims the dataset to appropriate data for the node
# Recurses from bottom to top
def data_values(node, dataFrame,right = None):
    
    # trim data starting from parent value
    if node.parent:
        dataFrame = data_values(node.parent,dataFrame,node.right)
        
    # if data is being split at node and the node has a split
    # Split data according to next node direction
    if right != None and node.children:
        
        feature, operator, value = node.name.split()
        return split_data(feature, operator, value, dataFrame)[int(right)]
    # If leaf or not splitting data at node
    # Return data at that node
    else:
        return dataFrame

# Verifies if all leaves in a tree have the minimum amount of data points
def verify_minimum_leaves(tree, n_min = 1):
    truth = True
    if tree.is_leaf:
        if tree.size < n_min:
            truth = False
    else:
        for child in tree.children:
            truth = truth == verify_minimum_leaves(child, n_min)
    
    return truth

############################################################

# Gets list of leaves of tree
def get_leaves(tree):
    leaves = []
    if not tree.is_leaf:
        for children in tree.children:
            leaves += get_leaves(children)
    else:
        leaves.append(tree)
    return leaves

# Creates dictionary of hashed leaves with values of 0
# Used for abstraction of leaves
def leaves_to_dict(tree):
    leaf_dict = {}
    leaves = get_leaves(tree)
    for leaf in leaves:
        leaf_dict[hash(leaf)] = 0
    return leaf_dict

#  Updates hashed leaf dictionary with data
def put_leaf_dict_data(tree, dataFrame, leafDict):
    # Update children of tree
    if not tree.is_leaf:
        
        # Get new data split for child nodes
        feature, operator, value = tree.name.split()
        leftData,rightData  = split_data(feature, operator, value, dataFrame)
        # Recure through all children
        if((not leftData.empty) and not tree.children[0].right):
            put_leaf_dict_data(tree.children[0],leftData,leafDict)
        if((not rightData.empty) and len(tree.children)>1):
            put_leaf_dict_data(tree.children[1],rightData,leafDict)
        elif((not rightData.empty) and tree.children[0].right):
            put_leaf_dict_data(tree.children[0],rightData,leafDict)
    else:
        leafDict[hash(tree)] = dataFrame['Class'].value_counts()

# Removes data values from hashed leaf dictionary
def minus_leaf_dict_data(tree, dataFrame, leafDict):
    # Update children of tree
    if not tree.is_leaf:
        
        # Get new data split for child nodes
        feature, operator, value = tree.name.split()
        leftData,rightData  = split_data(feature, operator, value, dataFrame)
        # Recure through all children
        if((not leftData.empty) and not tree.children[0].right):
            minus_leaf_dict_data(tree.children[0],leftData,leafDict)
        if((not rightData.empty) and len(tree.children)>1):
            minus_leaf_dict_data(tree.children[1],rightData,leafDict)
        elif((not rightData.empty) and tree.children[0].right):
            minus_leaf_dict_data(tree.children[0],rightData,leafDict)
    else:
        leafDict[hash(tree)] = leafDict[hash(tree)].subtract(dataFrame['Class'].value_counts(),fill_value = 0).sort_values(ascending = False)

# Adds data values to hashed leaf dictionary        
def plus_leaf_dict_data(tree, dataFrame, leafDict):
    # Update children of tree
    if not tree.is_leaf:
        
        # Get new data split for child nodes
        feature, operator, value = tree.name.split()
        leftData,rightData  = split_data(feature, operator, value, dataFrame)
        # Recure through all children
        if((not leftData.empty) and not tree.children[0].right):
            plus_leaf_dict_data(tree.children[0],leftData,leafDict)
        if((not rightData.empty) and len(tree.children)>1):
            plus_leaf_dict_data(tree.children[1],rightData,leafDict)
        elif((not rightData.empty) and tree.children[0].right):
            plus_leaf_dict_data(tree.children[0],rightData,leafDict)
    else:
        leafDict[hash(tree)] = dataFrame['Class'].value_counts().add(leafDict[hash(tree)],fill_value = 0).sort_values(ascending = False)

# Gets error of a sorted series (using most popular class)
def series_error(series):
    return series.sum() -  series.iloc[0] 

# Outputs error of series
def check_dict_error(item):
    if type(item) == pd.Series:
        return series_error(item)
    else:
        return 0

# Gets error of a hashed leaf dictionary
def get_dict_error(leafDict):
    error = 0
    for key in leafDict.keys():
        error += check_dict_error(leafDict[key])
    return error

# Debugging function for error of a hashed leaf dictionary
def get_each_dict_error(leafDict):
    for key in leafDict.keys():
        print(leafDict[key])
        print(check_dict_error(leafDict[key]))

# Exhaustively finds the local optimal data split of a node while keeping it's children
def optimal_node_data_split_dict(node, dataFrame, alpha = 0.5, baseLineError = 1, n_min = 1): 
    # Gets 
    childNodes = [copy.deepcopy(childNode) for childNode in node.children]
    bestError = [calculate_baseline_error(dataFrame),"","",""]
    tree_dict = leaves_to_dict(node)
    if(len(childNodes) != 1):
        if(len(childNodes) == 0):
            leftNode = Node("", right = False, checked = False)
            rightNode = Node("", right = True, checked = False)
        else:
            leftNode = childNodes[0]
            rightNode = childNodes[1]
        for feature in get_features(dataFrame):
            left_cumulative_error = []
            right_cumulative_error = []
            values = dataFrame[feature].drop_duplicates().sort_values().reset_index(drop=True)
            n = len(values.index)
            
            # For every feature, reset left and right dictionary
            left_leaf_dict = leaves_to_dict(leftNode)
            right_leaf_dict = leaves_to_dict(rightNode)
            put_leaf_dict_data(rightNode, dataFrame, right_leaf_dict)
            for i, point in enumerate(values[:-1]):
                samePoints = dataFrame.query('{feature} == {value}'.format(feature = feature,value = point))
                minus_leaf_dict_data(rightNode, samePoints, right_leaf_dict)
                plus_leaf_dict_data(leftNode, samePoints, left_leaf_dict)
                
                new_error = get_dict_error(left_leaf_dict) + get_dict_error(right_leaf_dict)
                if(new_error < bestError[0]):
                    bestError = [new_error,feature,"<",(1/2 * (values.iloc[i] + values.iloc[i+1])).round(2)]
                    # For debugging purposes
                    print(bestError)
        if(bestError[0] < calculate_baseline_error(dataFrame)):
            bestErrorNode = Node('{feature} {operator} {value}'.format(feature = bestError[1],operator = bestError[2],value = bestError[3]), children = childNodes, checked = False)
            if(childNodes):
                # Update children of replacement node at new decision point
                update_tree(bestErrorNode,dataFrame)
                #bestErrorNode.checked = True
            else:
                # Create children at new decision point for nodes which were previously leaves
                leftDataFrame,rightDataFrame = split_data(bestError[1],"<",bestError[3],dataFrame)
                create_leaf(leftDataFrame,parent=bestErrorNode,right=False)
                create_leaf(rightDataFrame,parent=bestErrorNode,right=True)
        else:
            bestErrorNode = node
        return bestErrorNode
    else:
        return delete_split(node, dataFrame, alpha, baseLineError)
    
#######

def place_left_node(node):
    if (not node.right and node.parent):
        if(len(node.parent.children)>1):
            node.parent.children = [node.parent.children[1],node.parent.children[0]]
    
# Create new split at given node
def optimal_split(node, dataFrame, alpha = 0.5, baseLineError = 1, n_min = 1):
    
    # Extract set of features
    features = get_features(dataFrame)
    
    # Create new optimal split if it does not violate minimum number of data points
    if(len(dataFrame.index)) >= n_min * 2:
        
        # Create parallel split
        tree = optimal_node_data_split_dict(node, dataFrame, alpha = alpha, baseLineError = baseLineError, n_min = n_min)
        # Add new split node to tree
        parent = node.parent
        right = node.right
        tree.right = right
        tree.parent = parent
        
        # Remove old split node from tree
        node.parent = None
        # Place left nodes back on left
        place_left_node(tree)
        
        # change node to decision
        return tree
    else:
        return node

# Updates tree with new data values
def update_tree(tree, dataFrame, n_min = 1):
    # Reset checked flag
    tree.checked = False
    
    # Update children of tree
    if not tree.is_leaf:
        
        # Get new data split for child nodes
        feature, operator, value = tree.name.split()
        leftData,rightData  = split_data(feature, operator, value, dataFrame)
        
        # Recure through all children
        if((not leftData.empty) and not tree.children[0].right):
            update_tree(tree.children[0],leftData)
        if((not rightData.empty) and len(tree.children)>1):
            update_tree(tree.children[1],rightData)
        elif((not rightData.empty) and tree.children[0].right):
            update_tree(tree.children[0],rightData)
    else:
        # debugging
        if(len(dataFrame.index) == 0):
            print(" No items at tree: ", tree)
        
        
        # Update leaf for new data points
        Class = dataFrame['Class'].value_counts().first_valid_index()
        tree.error = calculate_combined_error(dataFrame, Class)
        tree.size = len(dataFrame.index)
        tree.accuracy =  (dataFrame['Class'].value_counts().iloc[0]/len(dataFrame.index)).round(3)
        tree.name = "class = {Class}, accuracy = {accuracy}, size = {size} error = {error}".format(Class=Class,accuracy=tree.accuracy,size = tree.size, error = tree.error)
        tree.Class =  Class

    
# Delete the split at the given node
def delete_split(node, dataFrame, alpha = 0.5, baseLineError = 1):
    # Set default node to original node
    newNode = node
    
    # Check if node has children
    if node.children:
        
        # If multiple children
        if len(node.children) > 1:
            
            # Try replacing with left child
            leftNode = copy.deepcopy(node.children[0])
            
            # Try replacing with right child
            rightNode = copy.deepcopy(node.children[1])
            
            
            # Get loss of original node
            currentLoss = loss_function(node, dataFrame, alpha, baseLineError)
            
            # Update children of left and right node with new data
            # Try catch is for debugging only
            try:
                update_tree(leftNode,dataFrame)
            except:
                print(20*"Big Error \n")
                print_tree(get_root(leftNode))
                print("leftNode:", leftNode)
                
                print("node", node)
                print(dataFrame)
                
            try:
                update_tree(rightNode,dataFrame)
            except:
                print(20*"Big Error \n")
                print_tree(get_root(rightNode))
                print("rightNode:", rightNode)
                
                print("node", node)
                print(dataFrame)
            
            # Get loss of new updated left and right trees
            rightLoss = loss_function(rightNode, dataFrame, alpha, baseLineError)
            leftLoss = loss_function(leftNode, dataFrame, alpha, baseLineError)
            
            # Replace old node with left or right node if they have lower loss
            if leftLoss < rightLoss:
                if leftLoss < currentLoss:
                    newNode = copy.deepcopy(leftNode)
            else:
                if rightLoss < currentLoss:
                    newNode = copy.deepcopy(rightNode)
            
            
            
            if newNode != node:
                
                # Add new node to tree
                newNode.parent = node.parent
                
                # get the position of the node relative to the parent
                newNode.right = node.right
                
                # Remove original node from tree
                node.parent = None
                
                # Reset left positioned nodes
                place_left_node(newNode)
                
        # Single child
        else:
            newNode = node.children[0]
            
    return newNode

# Create a new split at the node
def update_split(node, dataFrame, alpha = 0.5, baseLineError = 1, n_min = 1):
    # replace previous split with new optimal split
    return optimal_split(node,dataFrame, alpha, baseLineError, n_min)

# Creates a new split at a leaf
def create_split(node, dataFrame, alpha = 0.5, baseLineError = 1, n_min = 1):
    
    newNode = node
    
    if node.is_leaf:
        
        # Only creates leaf if accuracy cannot be improved or improved node would violate minimum data count at leaves
        if node.accuracy < 1 and len(dataFrame.index) >= n_min * 2:
            
            newNode = optimal_split(node, dataFrame, alpha, baseLineError, n_min)
    
    # Incase non-leaves are run through create split
    else:
        newNode = optimal_split(node, dataFrame, alpha, baseLineError, n_min)
        
    return newNode
            
# Returns random unchecked node from tree
def get_random_node(tree):
    availableNodes = [node for node in PreOrderIter(tree) if not node.checked]
    #availableNodes = [tree.root]
    if availableNodes:
        return random.choice(availableNodes)
    else:
        return None

# Gets root of node
# If no root, returns node
def get_root(node):
    if(node.root):
        return node.root
    else:
        return node

# Calculates baseline error of a dataset
def calculate_baseline_error(dataFrame):
    return len(dataFrame.index) - dataFrame["Class"].value_counts().iloc[0]

# Unchecks all parents of a node recursively
def uncheck_parents(node):
    if node.parent:
        node.parent.checked = False
        uncheck_parents(node.parent)

# Replaces a tree with a deep copy of a tree from a given node and unchecks parents of given node
def replace_tree(newNode):
    uncheck_parents(newNode)
    return copy.deepcopy(get_root(newNode)), True

# Main loop for creating Optimal Classification Trees
def random_node_modification(tree, dataFrame, n_min = 1, D_max = 5, alpha = 0.5):
    
    # Calculate baseline error of data
    baseLineError = calculate_baseline_error(dataFrame)
    
    # Loops through while unchecked nodes are in the tree
    while get_random_node(tree):
        # Set improvement check
        improvement = False
        
        # Pick random unchecked node for modification
        node = get_random_node(tree)
        
        # Get data frame at specific node
        dataFrameNode = data_values(node,dataFrame)
        
        # Get loss of picked node
        loss = loss_function(node, dataFrameNode, alpha, baseLineError)
        
        
        
        # Runs update (parallel) split and delete split on decision nodes
        if not node.is_leaf:
            
            # Creates deep copy of node to improve
            testNode = copy.deepcopy(node)
            
            # Tries replacing node with one of it's children
            newNode = delete_split(testNode, dataFrameNode, alpha, baseLineError)
            
            # Calculates loss for new updated tree
            newLoss = loss_function(newNode,dataFrameNode, alpha, baseLineError)
            
            # Replaces original node if new node is better
            if newLoss < loss:
                
                tree, improvement =  replace_tree(newNode)
                print("Deleted split: \n", newNode)
                print_tree(tree)
                # Update loss in case optimal split overrides
                loss = loss_function(newNode, dataFrameNode, alpha, baseLineError)
                testNode = copy.deepcopy(newNode)
            else:
                # Creates deep copy of node to improve
                testNode = copy.deepcopy(node)
            
            # Tries replacing node with a parallel split node
            newNode = update_split(testNode, dataFrameNode, alpha, baseLineError, n_min)
            
            # Calculates loss for new updated tree
            newLoss = loss_function(newNode,dataFrameNode, alpha, baseLineError)
            
            # Replaces original node if new node is better
            if newLoss < loss:
                print("Updated split: \n", newNode)
                tree, improvement =  replace_tree(newNode)
                print_tree(tree)
        
        else:
            
            # Only creates new split if leaf is not at max depth
            if node.depth < D_max:
                # Creates deep copy of node to improve
                testNode = copy.deepcopy(node)
                
                # Tries creating new node to replace leaf
                newNode = create_split(testNode, dataFrameNode, alpha, baseLineError, n_min)
                
                # Calculates loss for new updated tree
                newLoss = loss_function(newNode, dataFrameNode, alpha, baseLineError)
                
                # Replaces original node if new node is better
                if newLoss < loss:
                    print("Created split: \n", newNode)
                    tree, improvement =  replace_tree(newNode)
                    print_tree(tree)
        
        # Check the node if not improved
        if not improvement:
            node.checked = True
        
        # Reset Improvement check
        improvement = False
        
    return tree


In [52]:
# Finds best local optimum tree with given hyperparameters and tree count
def best_local_optimum_tree(dataFrame, n = 1, n_min = 1, D_max = 5, alpha = 0.5):
    
    # Initialse baseline
    baseLineError = calculate_baseline_error(dataFrame)
    
    # Intiliase error for baseline leaf
    optimalTree = create_leaf(dataFrame)
    bestLoss = loss_function(optimalTree, dataFrame, alpha, baseLineError)
    print(bestLoss)
    # Loop through number of trees
    for i in range(0,n):
        print("Starting tree: ", i)
        # Create random tree
        startingTree = create_tree(dataFrame, get_features(dataFrame), remainingLength = D_max - 1, n_min = n_min)
        print_tree(startingTree)
        # Run optimal tree function to find local optimum tree
        localOptimalTree = random_node_modification(startingTree,test_data,alpha = alpha, D_max = D_max, n_min = n_min)
        
        # Get loss of local optimum tree
        localLoss = loss_function(localOptimalTree, dataFrame, alpha, baseLineError)
        print_tree(localOptimalTree)
        print("Error: ",localLoss)
        # Replace best tree with new local optimum tree if better
        if(localLoss < bestLoss):
            print_tree(localOptimalTree)
            optimalTree = copy.deepcopy(localOptimalTree)
            print_tree(optimalTree)
            # Update best loss
            bestLoss = localLoss
    
    return optimalTree
#optimalTree = best_local_optimum_tree(test_data, 5, 1, 5, 0.01)
#print_tree(optimalTree)

In [ ]:
def optimal_node_data_split(node, dataFrame, operators=["<"], alpha = 0.5, baseLineError = 1, n_min = 1): 
    print_tree(node)
    #node, dataFrame, operators=["<"], n_min = 1, alpha = 0.5, baseLineError = 1
    childNodes = [copy.deepcopy(childNode) for childNode in node.children]
    bestError = [np.inf,"","",""]
    print(dataFrame)
    if(len(childNodes)==2):
        leftNode = childNodes[0]
        rightNode = childNodes[1]
        
        for feature in get_features(dataFrame):
            left_cumulative_error = []
            right_cumulative_error = []
            values = dataFrame[feature].drop_duplicates().sort_values().reset_index(drop=True)
            n = len(values.index)
            for i,point in enumerate(values):
                
                samePoints = dataFrame.query('{feature} == {value}'.format(feature = feature,value = point))
                print(samePoints)
                print(point,error_value(leftNode,samePoints),error_value(rightNode,samePoints))
                
                #Replace "error_value" with function that can manipulate 
                if(i==0):
                    left_cumulative_error.append(error_value(leftNode,samePoints))
                    right_cumulative_error.append(error_value(rightNode,samePoints))
                else:
                    left_cumulative_error.append(error_value(leftNode,samePoints) + left_cumulative_error[-1])
                    right_cumulative_error.insert(0,error_value(rightNode,samePoints) + right_cumulative_error[0])
            print(feature)
            print(left_cumulative_error)
            print(right_cumulative_error)
            if(len(left_cumulative_error) == len(right_cumulative_error)):
                for i in range(0,len(left_cumulative_error)):
                    new_error = left_cumulative_error[i] + right_cumulative_error[i]
                    if(new_error < bestError[0]):
                            
                        bestError = [new_error,feature,"<",(1/2 * (values.iloc[i] + values.iloc[i+1]))]
                        print(bestError)
        bestErrorNode = Node('{feature} {operator} {value}'.format(feature = bestError[1],operator = bestError[2],value = bestError[3], children = childNodes, checked = True))
        if(childNodes):
            # Update children of replacement node at new decision point
            update_tree(bestErrorNode,dataFrame)
            bestErrorNode.checked = True
        else:
            # Create children at new decision point for nodes which were previously leaves
            leftDataFrame,rightDataFrame = split_data(bestError[1],"<",bestError[3],dataFrame)
            create_leaf(leftDataFrame,parent=bestErrorNode,right=False)
            create_leaf(rightDataFrame,parent=bestErrorNode,right=True)
        return bestErrorNode
    else:
        return delete_split(node, dataFrame, alpha, baseLineError)

In [21]:
test_data.query("PetalWidthCm < 1.65 and PetalLengthCm >= 2.45  and PetalLengthCm < 4.95")

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Class
50,7.0,3.2,4.7,1.4,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
53,5.5,2.3,4.0,1.3,Iris-versicolor
54,6.5,2.8,4.6,1.5,Iris-versicolor
55,5.7,2.8,4.5,1.3,Iris-versicolor
56,6.3,3.3,4.7,1.6,Iris-versicolor
57,4.9,2.4,3.3,1.0,Iris-versicolor
58,6.6,2.9,4.6,1.3,Iris-versicolor
59,5.2,2.7,3.9,1.4,Iris-versicolor
